# Some observations on the data below:
* People with age above 50 and avg_gulucose_level above 160 and married have a high chance of stroke
* Random Forest and XgBoost are the best classifiers

In [ ]:
### Imports

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

### Pre Process Data

In [ ]:
#df =  pd.read_csv('healthcare-dataset-stroke-data.csv')
df =  pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isna().sum().plot(kind = 'bar')

In [ ]:
# Replace null values with mean
df.bmi.replace(to_replace=np.nan, value=df.bmi.mean(), inplace=True)

In [ ]:
df.describe().T

## EDA

In [ ]:
categorical_columns = df.dtypes[df.dtypes == 'object'].index
numerical_columns = df.dtypes[df.dtypes != 'object'].index

In [ ]:
categorical_columns

In [ ]:
df['gender'].value_counts()

In [ ]:
df['work_type'].value_counts().plot(kind = 'bar')

In [ ]:
df['ever_married'].value_counts()

In [ ]:
df['Residence_type'].value_counts()

In [ ]:
df['smoking_status'].value_counts().plot(kind = 'bar')

In [ ]:
df['hypertension'].value_counts()

In [ ]:
df['heart_disease'].value_counts()

In [ ]:
numerical_columns

In [ ]:
#df['age'].hist()
sns.distplot(df['age'])

In [ ]:
#df['bmi'].hist()
sns.distplot(df['bmi'])

In [ ]:
sns.distplot(df['avg_glucose_level'])

In [ ]:
sns.scatterplot(x = 'avg_glucose_level',y = 'bmi',hue = 'stroke', size = 'heart_disease', data =df)

In [ ]:
sns.lineplot(x = 'age',y = 'avg_glucose_level',hue = 'stroke',size='gender', data =df)

In [ ]:
sns.catplot(x='heart_disease',y='age', hue="work_type", kind="bar", data=df)

In [ ]:
sns.catplot(x='stroke', y="age", hue = 'work_type', kind="box", data=df)

In [ ]:
# Heat Map Correlation 
# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
plt.figure(figsize=(12,10))

sns.distplot(df[df['stroke'] == 0]["bmi"], color='green') # No Stroke - green
sns.distplot(df[df['stroke'] == 1]["bmi"], color='red') # Stroke - Red

plt.title('No Stroke vs Stroke by BMI', fontsize=15)
plt.xlim([10,100])
plt.show()

In [ ]:
plt.figure(figsize=(13,13))
sns.set_theme(style="darkgrid")
plt.subplot(2,3,1)
sns.violinplot(x = 'gender', y = 'stroke', data = df)
plt.subplot(2,3,2)
sns.violinplot(x = 'hypertension', y = 'stroke', data = df)
plt.subplot(2,3,3)
sns.violinplot(x = 'heart_disease', y = 'stroke', data = df)
plt.subplot(2,3,4)
sns.violinplot(x = 'ever_married', y = 'stroke', data = df)
plt.subplot(2,3,5)
sns.violinplot(x = 'work_type', y = 'stroke', data = df)
plt.xticks(fontsize=9, rotation=45)
plt.subplot(2,3,6)
sns.violinplot(x = 'Residence_type', y = 'stroke', data = df)
plt.show()

#### Target value

In [ ]:
df['stroke'].value_counts()

## Data Preprocessing 

In [ ]:
X = df.iloc[:, 1:-1].values
y = df.iloc[:, -1].values

### Categorical Encoding 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ct = ColumnTransformer(transformers= [('encoder', OneHotEncoder(), [0,5,9])], remainder= 'passthrough')
X = np.array(ct.fit_transform(X))

### Label Encoding 
columns: 'ever_married' and 'residence_type'

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
X[:, 15] = le.fit_transform(X[:, 15])
X[:, 16] = le.fit_transform(X[:, 16])

### TREATING IMBALANCE CLASS USING SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote = SMOTE()
x_smote, y_smote = smote.fit_resample(X, y)

### SPLITTING OF DATASET INTO TRAIN AND TEST

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test, y_train,y_test=train_test_split(x_smote,y_smote,test_size=0.33,random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
print("Number transactions x_train dataset: ", X_train.shape)
print("Number transactions y_train dataset: ", y_train.shape)
print("Number transactions x_valid dataset: ", X_valid.shape)
print("Number transactions y_valid dataset: ", y_valid.shape)
print("Number transactions x_test dataset: ", X_test.shape)
print("Number transactions y_test dataset: ", y_test.shape)

### STANDARDIZATION 

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sc=StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
X_valid = sc.transform(X_valid)

## Model Building

### Logistic Regression as a benchmark model

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

In [ ]:
y_pred_log_reg_valid = log_reg.predict(X_valid)
# y_pred_log_reg

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score

In [ ]:
classification_report = classification_report(y_valid, y_pred_log_reg_valid)
print(classification_report)

In [ ]:
auc = roc_auc_score(y_valid, y_pred_log_reg_valid)
auc

In [ ]:
cm = confusion_matrix(y_valid, y_pred_log_reg_valid)
cm

#### ROC CURVE

In [ ]:
predicted_probab_log = log_reg.predict_proba(X_valid)
predicted_probab_log = predicted_probab_log[:, 1]
fpr, tpr, _ = roc_curve(y_valid, predicted_probab_log)

In [ ]:
from matplotlib import pyplot
pyplot.plot(fpr, tpr, marker='.', label='Logistic Regression')
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
pyplot.legend()
pyplot.show()

### Other models

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
models = []
models.append(['SVM', SVC(random_state=0)])
models.append(['KNeighbors', KNeighborsClassifier()])
models.append(['GaussianNB', GaussianNB()])
models.append(['BernoulliNB', BernoulliNB()])
models.append(['Decision Tree', DecisionTreeClassifier(random_state=0)])
models.append(['Random Forest', RandomForestClassifier(random_state=0)])
models.append(['XGBoost', XGBClassifier(eval_metric= 'error')])

In [ ]:
lst_1= []
for m in range(len(models)):
    lst_2= []
    model = models[m][1]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    cm = confusion_matrix(y_valid, y_pred)  #Confusion Matrix
    accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10)   #K-Fold Validation
    roc = roc_auc_score(y_valid, y_pred)  #ROC AUC Score
    precision = precision_score(y_valid, y_pred)  #Precision Score
    recall = recall_score(y_valid, y_pred)  #Recall Score
    f1 = f1_score(y_valid, y_pred)  #F1 Score

    lst_2.append(models[m][0])
    lst_2.append((accuracy_score(y_valid, y_pred))*100) 
    lst_2.append(accuracies.mean()*100)
    lst_2.append(accuracies.std()*100)
    lst_2.append(roc)
    lst_2.append(precision)
    lst_2.append(recall)
    lst_2.append(f1)
    lst_1.append(lst_2)

In [ ]:
df = pd.DataFrame(lst_1, columns= ['Model', 'Accuracy', 'K-Fold Mean Accuracy', 'Std. Deviation', 'ROC AUC', 'Precision', 'Recall', 'F1'])

In [ ]:
df.sort_values(by= ['Accuracy', 'K-Fold Mean Accuracy'], inplace= True, ascending= False)
df

#### As we can see that Random Forest and XgBoost works best we will hyper tune their parameters

#### Hyperparameter Tuning using Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_models = [(RandomForestClassifier(),[{'n_estimators':[100,150,200],'criterion':['gini','entropy'],'random_state':[0]}]), 
              (XGBClassifier(), [{'learning_rate': [0.01, 0.05, 0.1], 'eval_metric': ['error']}])]

In [ ]:
for i,j in grid_models:
    grid = GridSearchCV(estimator=i,param_grid = j, scoring = 'accuracy',cv = 10)
    grid.fit(X_train, y_train)
    best_accuracy = grid.best_score_
    best_param = grid.best_params_
    print('{}:\nBest Accuracy : {:.2f}%'.format(i,best_accuracy*100))
    print('Best Parameters : ',best_param)
    print('')
    print('----------------')
    print('')

### RandomForest and evaluating on test set

In [ ]:
classifier = RandomForestClassifier(criterion= 'entropy', n_estimators= 150, random_state= 0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]

In [ ]:
print(f'ROC AUC score: {roc_auc_score(y_test, y_prob)}')
print('Accuracy Score: ',accuracy_score(y_test, y_pred))

In [ ]:
# print(classification_report(y_test, y_pred))

In [ ]:
# Visualizing Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (8, 5))
sns.heatmap(cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15}, 
            yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

In [ ]:
from sklearn import metrics

In [ ]:
# Roc AUC Curve
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_test, y_prob)
roc_auc = metrics.auc(false_positive_rate, true_positive_rate)

In [ ]:
sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

### XgBoost Classifier and evaluating on test set

In [ ]:
classifier = XGBClassifier(eval_metric= 'error', learning_rate= 0.1)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
y_prob = classifier.predict_proba(X_test)[:,1]

In [ ]:
# print(classification_report(y_test, y_pred))
print(f'ROC AUC score: {roc_auc_score(y_test, y_prob)}')
print('Accuracy Score: ',accuracy_score(y_test, y_pred))

In [ ]:
# Visualizing Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize = (8, 5))
sns.heatmap(cm, cmap = 'Blues', annot = True, fmt = 'd', linewidths = 5, cbar = False, annot_kws = {'fontsize': 15}, 
            yticklabels = ['No stroke', 'Stroke'], xticklabels = ['Predicted no stroke', 'Predicted stroke'])
plt.yticks(rotation = 0)
plt.show()

In [ ]:
# Roc Curve
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_test, y_prob)
roc_auc = metrics.auc(false_positive_rate, true_positive_rate)

In [ ]:
sns.set_theme(style = 'white')
plt.figure(figsize = (8, 8))
plt.plot(false_positive_rate,true_positive_rate, color = '#b01717', label = 'AUC = %0.3f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1], linestyle = '--', color = '#174ab0')
plt.axis('tight')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Both models are probably overfitting the data. We need to add some Regularization here